##  First go at a real SAR Processor

###  Eric Sutherland
### 2/15/2024

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from scipy.signal import chirp
from scipy.interpolate import interp1d
from simplekml import Kml
import time
import re
import pandas as pd

import functions as fun
import novatel_functions as nov


# %matplotlib widget

params = {"ytick.color" : "black",
          "xtick.color" : "black",
          "axes.labelcolor" : "black",
          "axes.edgecolor" : "black",
          "font.family" : "serif",
          "font.serif" : ["times new roman"],
          "figure.dpi" : 150}
plt.rcParams.update(params)

%matplotlib inline

# Important constants that change with datasets #

In [ ]:

###  Number of files and how much data to import  ###
start_file = 640#250 # 600
stop_file = 660#270  # 680
start_sample = 000
stop_sample = 40000

num_files = stop_file - start_file
# stop_sample = 24000

## Platform attitude basics  ##
# vp = 72  ## Platform velocity in [m/s]
# H = 2000  ## Platform height [m]
# theta_ = np.radians(45)  ## estimated look angle
# bw_el = np.radians(45)  ##  estimating elevation beamwidth
# swath_width = 10000  ## desired swath width in [m]

vp = 72  ## Platform velocity in [m/s]
H = 1900  ## Platform height [m]
theta_ = np.radians(45)  ## estimated look angle
bw_el = np.radians(45)  ##  estimating elevation beamwidth
swath_width = 10000  ## desired swath width in [m]

## Constants  ##
c = 3e8
num_plots = 25  ## helps to determine how many lines to plot (fewer speeds things up)

##  radar parameters  ##
f0 = 5.39e9
# f0 = 13.64e9
prf = 1e3
lambda_ = c/f0


## Compression parameters  ##
fs = 491.52e6*2

###  For NH Data  ###
# fl = 240e6
# fh = 320e6
# tp = 5.7e-6

###  For CO Data  ###
fl = 143e6
fh = 223e6
tp = 11e-6


chirp_direction = 'up'
B = fh - fl



In [ ]:
directory = r'\\seasat\Projects\SNOWWI\SNOWWI_NH\NH_Flight_December\20231215T144359'  ####  CBand in NH over airport
# directory = r'\\seasat\Projects\SNOWWI\SNOWWI_NH\NH_Flight_December\20231215T142814'  ### CBand NH december
# directory = r'\\seasat\Projects\SNOWWI\Colorado_January\20240205T020242\chan0'  ####  CBand in Grand Junction
# directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Radar_data\from_disk_F\save_data_nvme1n1\20240327T120951\chan0'  ### CBand CO March
# directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Radar_data\from_disk_F\save_data_nvme1n1\20240327T124951\chan0'  ### CBand CO March
directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Radar_data\from_disk_F\save_data_nvme1n1\20240327T123855\chan2'  ### CBand CO March
# directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Radar_data\from_disk_F\save_data_nvme1n1\20240327T125534\chan2'


# directory = r'\\seasat\Projects\SNOWWI\Colorado_January\20240204T193523\chan0'  ####  Ku Low in Grand Junction

# directory = r'\\Sentinel\SNOWWI\Colorado2024\January\Radar_data\save_data_nvme0\20240205T031232\chan0'  ####  CBand in Grand Mesa


filelist = fun.list_files(directory)

rawdata = fun.load_data(filelist, start_file, stop_file, start_sample=start_sample, stop_sample=stop_sample)



In [ ]:


# novatel_directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Novatel'
# novatel_filename = r'gm_20240327.txt'

# header = nov.get_header(filelist[start_file])
# unix_start_time = nov.timestamp_from_header(header) + 7200 + 120

# gps_to_unix = 315964800

# gps_start_time = unix_start_time - gps_to_unix
# gps_stop_time = gps_start_time + num_files/2

# novatel = nov.readNovatel(novatel_directory, novatel_filename, gps_start_time, gps_stop_time)


In [ ]:
# novatel_directory = r'\\Sentinel\SNOWWI\Colorado2024\March\Novatel'
# novatel_filename = r'gm_20240327.txt'

# flightline = nov.readNovatel(novatel_directory, novatel_filename, gps_start_time, gps_stop_time)
# base_time, base_dt = nov.get_time(flightline)

# yaw, pitch, roll = nov.get_yaw_pitch_roll(flightline)
# yaw, pitch, roll, time = nov.interpolate_attitude(yaw, pitch, roll, base_time, base_dt, rawdata.shape[0])

# x, y, z = nov.get_xyz(flightline)
# xfit, yfit, zfit = nov.fit_position(x, y, z)

# xfit, yfit, zfit, _ = nov.interpolate_xyz(xfit, yfit, zfit, base_time, base_dt, rawdata.shape[0])

# dt = 1/prf


In [ ]:
# dx = np.diff(xfit)
# dy = np.diff(yfit)
# dz = np.diff(zfit)

# ds = np.sqrt(dx**2 + dy**2 + dz**2)
# s = np.cumsum(ds)
# vp = ds/dt

# vp_mean = np.mean(vp)

In [ ]:
###  Plotting raw radar data  ###
%matplotlib widget

n = int(rawdata.shape[0]/num_plots)

plt.figure(figsize=(16, 4))

for i in rawdata[0::n]:
    plt.plot(i)



plt.title(f'{num_plots} Raw Pulses - Sanity Check')
plt.xlabel('Range Samples')
plt.ylabel('Magnitude Samples')

plt.xlim(0, rawdata.shape[1])

plt.grid()
plt.show()

In [ ]:
# flatdata = rawdata.flatten()[19200:-(64000-19200)]
# reshaped = flatdata.reshape(-1, 64000)

In [ ]:
# rawdata = reshaped[:, 0:40000]
# del(reshaped)

In [ ]:
rawdata -= rawdata.mean(axis=1, ).reshape(-1, 1)

In [ ]:
###  Compressing and plotting compression waveforms  ###

rawdata = fun.bandpass(rawdata, fl-10e6, fh+10e6, fs, 3)
compressed = fun.compress(rawdata, tp, fs, fl, fh, chirp_direction, plot=False)

print(rawdata.dtype)
del(rawdata)

In [ ]:
### Truncate swath to be just returns  ###

# swath, Rmin, Rmax = truncate_swath(compressed, swath_width, H, tp, fs, theta_, bw_el)
# swath = compressed[:, start_sample+30000:]
swath = compressed[:, start_sample+21500:]
# swath = compressed[:, 19300:]
del(compressed)



In [ ]:
plt.figure()

plt.imshow(20*np.log10(abs(fun.multilook(swath, 2, 6))), vmin=70, vmax=100, origin='upper', cmap='gray', interpolation='bilinear', aspect='equal')#, extent=[Rmin/1000, Rmax/1000, azR/1000, 0])
# plt.imshow(np.angle(focused_rd[6000:7350, 7100:7500]), origin='lower', cmap='hsv', interpolation='None')
# plt.imshow(np.angle(range_corrected_rd[6000:7350, 7100:7500]), origin='lower', cmap='hsv', interpolation='None')
plt.title('Range Compressed')
plt.xlabel('Range Samples')
plt.ylabel('Azimuth Samples')
plt.colorbar(label='[dB - Uncal]')

# image_file = f'CBand_NH_Test2.png'
# cwd = os.getcwd()

# plt.savefig(os.path.join(cwd, image_file), dpi=1000, bbox_inches='tight', pad_inches=0.25, transparent=False)



plt.show()

In [ ]:
# Rmin = flightline['H-Ell'].mean() - (10000/3.25)
Rmin = H
Rmax = Rmin + ((c/fs)*swath.shape[1])/2

print(Rmin)
print(Rmax)

In [ ]:
# print(swath.shape)
# print(s.shape)

###  Resample each range line in azimuth  ###

In [ ]:
# alongtrack = np.linspace(s[0], s[-1], len(s))

# for i in range(swath.shape[1]):
#     interpolator = interp1d(s, swath[:, i], kind='cubic', fill_value='extrapolate')
#     swath[:, i] = interpolator(alongtrack)

In [ ]:
x = [1, 2, 3, 4]

print(x[:3])

In [ ]:
###  Plotting image of compressed data

# %matplotlib inline

# n = int(swath.shape[0]/num_plots)



# fig, ax = plt.subplots(figsize=[16, 4], nrows=1, ncols=2)

# im = ax[0].imshow(20*np.log10(abs(swath)), vmin=90, vmax=120, origin='lower', interpolation='None', cmap='gray', aspect='equal')
# ax[0].set_title('Compressed Data')
# ax[0].set_xlabel('Range Samples')
# ax[0].set_ylabel('Azimuth Samples')
# fig.colorbar(im, ax=ax[0], label='[dB]')

# for i in swath[::n]:
#     ax[1].plot(20*np.log10(abs(i)), alpha=0.3)

# ax[1].set_title('Compressed Range Lines')
# ax[1].set_xlabel('Range Samples')
# ax[1].set_ylabel('[dB]')
# ax[1].set_xlim(0, swath.shape[1])
# ax[1].grid()

# plt.show()

In [ ]:
# %matplotlib widget

# plt.figure()

# plt.imshow(20*np.log10(abs(swath)), vmin=90, vmax=120, origin='upper', interpolation='None', cmap='gray', aspect='equal')

# image_file = f'compressed1.png'
# cwd = os.getcwd()

# plt.savefig(os.path.join(cwd, image_file), dpi=1000, bbox_inches='tight', pad_inches=0, transparent=True)

# plt.colorbar()
# plt.show()

## Azimuth FFT  ##

In [ ]:
###  Finding Azimuth FFT  ###

# swath = multilook(swath, 4, 1)

az_samples = swath.shape[0]
T_az = 1/prf
az_tmin = 0
az_tmax = T_az*az_samples

t_az = np.linspace(az_tmin, az_tmax, az_samples)
az_fft_data = np.fft.fftshift(np.fft.fft(swath, axis=0), axes=0).astype(np.complex64)
# az_fft_data = np.fft.fft(swath, axis=0).astype(np.complex64)



az_freq = np.linspace(-prf/2, prf/2, az_samples)
freq_khz = az_freq/1e3
print(az_fft_data.dtype)
# del(swath)


In [ ]:
# ###  Plotting some Azimuth FFT Stuffs  ###

vmin = 130
vmax = 150

# vmin = 140
# vmax = 160

n = int(az_fft_data.shape[1]/num_plots)

fig, ax = plt.subplots(figsize=[8, 4], nrows=1, ncols=2)

im = ax[0].imshow(20*np.log10(abs(az_fft_data)), vmin=vmin, vmax=vmax, origin='lower', interpolation='None', extent=[0, az_fft_data.shape[1], -prf/2, prf/2], aspect='auto')
ax[0].set_title('Azimuth Frequency Spectrogram')
ax[0].set_xlabel('Range Samples')
ax[0].set_ylabel('Azimuth Frequency [Hz]')
fig.colorbar(im, ax=ax[0], label='[dB]')

for i in az_fft_data.T[::n]:
    ax[1].plot(az_freq, 20*np.log10(fun.rolling_avg(abs(i), 100)), alpha=0.3)

ax[1].set_title('Azimuth Frequency of Different Range Lines')
ax[1].set_xlabel('Azimuth Frequency [Hz]')
ax[1].set_ylabel('[dB]')
ax[1].set_xlim(-prf/2, prf/2)
ax[1].grid()

plt.show()

###  Applying Range Cell Migration Corrections  ###

In [ ]:
range_corrected_rd = fun.rcmc(az_fft_data, lambda_, prf, fs, Rmin, Rmax, vp).astype(np.complex64)
print(range_corrected_rd.dtype)
del(az_fft_data)

## Smoothing Doppler, finding Doppler Centroids  ##

In [ ]:
# def fit_doppler(data, prf, fs, B, order, snr=True):


#     smoothed_doppler, range_corrected_rd = fun.smooth_doppler(data, fs, B)  ##  Running some rolling averages over azimuth doppler centroids
#     az_freq = np.linspace(-prf/2, prf/2, data.shape[0])

#     doppler_centroid_idx = np.argmax(smoothed_doppler, axis=0)

#     if snr == True:
#         means = np.mean(abs(smoothed_doppler), axis=0)
#         maxs = np.max(abs(smoothed_doppler), axis=0)

#         badsnr = np.where(maxs<(1.3*means))[0]  ## Finding indeces where snr < ...dB

#         doppler_centroid_idx[badsnr[0]:] = doppler_centroid_idx[badsnr[0] - 1]  ## replacing indeces with bad snr to the last index with a good snr
    
#     doppler_centroids = az_freq[doppler_centroid_idx]

#     range_samples = np.linspace(0, doppler_centroids.shape[0]-1, doppler_centroids.shape[0])

#     coeff = np.polyfit(range_samples, doppler_centroids, 1)
#     doppler_fit = coeff[0]*range_samples + coeff[1]  # **3 + coeff[1]*range_samples**2 + coeff[2]*range_samples + coeff[3]

#     return doppler_fit, range_corrected_rd, doppler_centroids




In [ ]:
# doppler_fit, test, doppler_centroids = fun.fit_doppler(range_corrected_rd, prf, fs, B, 3, False)
doppler_fit, doppler_centroids = fun.fit_doppler(range_corrected_rd, prf, fs, B, 1, False)
# doppler_fit = fun.load_doppler_fit(r'C:\Users\epsutherland\Desktop\ForThesis\GM_CReflectors\GM_CornerReflector_Doppler.dat')



print(doppler_fit.shape)
# print(doppler_centroids.dtype)

In [ ]:
# Rmin = H
# Rmax = Rmin + ((c/fs)*range_corrected_rd.shape[1])/2

In [ ]:
plt.figure()

plt.plot(doppler_fit)
# plt.plot(doppler_centroids)

plt.show()

In [ ]:
###  Plotting some Azimuth FFT Stuffs  ###

# vmin = 110
# vmax = 130

# # vmin = 140
# # vmax = 160

# n = int(range_corrected_rd.shape[1]/num_plots)

# plt.figure()

# im = plt.imshow(20*np.log10(abs(range_corrected_rd)), vmin=vmin, vmax=vmax, origin='lower', interpolation='None', extent=[0, range_corrected_rd.shape[1], -prf/2, prf/2], aspect='auto')
# plt.plot(doppler_fit, color='r', linewidth=0.8)
# plt.plot(doppler_centroids, color='k', linewidth=0.8)
# plt.title('Range Corrected Azimuth Frequency Spectrogram')
# plt.xlabel('Range Samples')
# plt.ylabel('Azimuth Frequency [Hz]')
# plt.colorbar(im, label='[dB]')

# # for i in range_corrected_rd.T[::n]:
# #     ax[1].plot(20*np.log10(abs(i)), alpha=0.3)

# # ax[1].set_title('Azimuth Frequency of Different Range Lines')
# # ax[1].set_xlabel('Azimuth Frequency [Hz]')
# # ax[1].set_ylabel('[dB]')
# # ax[1].set_xlim(-prf/2, prf/2)
# # ax[1].grid()

# plt.show()

In [ ]:
# smoothed_doppler, range_corrected_rd = smooth_doppler(range_corrected_rd, fs, B)  ##  Running some rolling averages over azimuth doppler centroids

# doppler_centroid_idx = np.argmax(smoothed_doppler, axis=0)
# doppler_centroids = az_freq[doppler_centroid_idx]

# range_samples = np.linspace(0, doppler_centroids.shape[0]-1, doppler_centroids.shape[0])

# coeff = np.polyfit(range_samples, doppler_centroids, 3)
# doppler_fit = coeff[0]*range_samples**3 + coeff[1]*range_samples**2 + coeff[2]*range_samples + coeff[3]

# doppler_fit = coeff[0]*range_samples**5 + coeff[1]*range_samples**4 + coeff[2]*range_samples**3 + coeff[3]*range_samples**2 + coeff[4]*range_samples + coeff[5]
# doppler_fit = coeff[0]*range_samples**2 + coeff[1]*range_samples + coeff[2]




In [ ]:
# def rcmc(data, lambda_, fs_az, fs_rng, Rmin, Rmax, vp):

#     az_samp = data.shape[0]
#     rng_samp = data.shape[1]

#     y = np.arange(az_samp)
#     x = np.arange(rng_samp)

#     fn = np.linspace(-fs_az/2, fs_az/2, az_samp)

#     # R0 = (Rmin + Rmax) / 2

#     R = np.linspace(Rmin, Rmax, rng_samp).reshape((-1, rng_samp))
#     fn = (np.linspace(-fs_az/2, fs_az/2, az_samp)**2).reshape((az_samp, -1))  ## squared because of range migration equation
#     Rfn = fn.dot(R)
#     dR = (lambda_**2 * Rfn) / (8*vp**2)

#     Rshift = (2*dR*fs_rng) / 3e8
#     # Rshift = x - np.array(shift_samples)[:, np.newaxis]

#     new_matrix = np.zeros_like(data)

#     # print(Rshift)
#     # return Rshift
#     xin = np.arange(rng_samp)
    
#     for i in range(az_samp):
#         x = xin - Rshift[i, :]
#         interp_func = interp1d(x, data[i], kind='cubic', bounds_error=False, fill_value=1)
#         new_matrix[i] = interp_func(np.arange(rng_samp))
    
#     return new_matrix


In [ ]:
# rd_corrections = rcmc(az_fft_data, lambda_, prf, Rmin, Rmax, vp)  ##  this matrix is the phase corrections that need to be applied
# range_corrected_rd = az_fft_data*rd_corrections




In [ ]:
# %matplotlib widget

plt.figure(figsize=(9, 4))
plt.imshow(20*np.log10(abs(range_corrected_rd)), vmin=130, vmax=150, origin='lower', cmap='viridis', interpolation='None')
plt.colorbar()
plt.show()

In [ ]:
###  Plotting some Azimuth FFT Stuffs  ###

# vmin = 130
# vmax = 150

# # vmin = 140
# # vmax = 160

# n = int(az_fft_data.shape[1]/num_plots)

# fig, ax = plt.subplots(figsize=[16, 4], nrows=1, ncols=2)

# im = ax[0].imshow(20*np.log10(abs(range_corrected_rd)), vmin=vmin, vmax=vmax, origin='lower', interpolation='None', extent=[0, az_fft_data.shape[1], -prf/2, prf/2], aspect='auto')
# ax[0].plot(doppler_fit, color='k')
# ax[0].set_title('Azimuth Frequency Spectrogram')
# ax[0].set_xlabel('Range Samples')
# ax[0].set_ylabel('Azimuth Frequency [Hz]')
# ax[0].grid()
# # fig.colorbar(im, ax=ax[0], label='[dB]')

# im = ax[1].imshow(20*np.log10(abs(smoothed_doppler)), vmin=vmin, vmax=vmax, origin='lower', interpolation='None', extent=[0, az_fft_data.shape[1], -prf/2, prf/2], aspect='auto')


# ax[1].set_title('Azimuth Frequency of Different Range Lines')
# ax[1].set_xlabel('Azimuth Frequency [Hz]')
# ax[1].set_ylabel('[dB]')
# # ax[1].set_xlim(-prf/2, prf/2)
# ax[1].grid()

# plt.show()

In [ ]:
# range_corrected_rd = range_corrected_rd[1:, :]
del(swath)

In [ ]:
R = np.linspace(Rmin, Rmax, range_corrected_rd.shape[1])

dR = (Rmax - Rmin)/range_corrected_rd.shape[1]
azR = dR*range_corrected_rd.shape[0]
# az = np.linspace(0, dR*range_corrected_rd.shape[0], range_corrected_rd.shape[0])


# Rmax = Rmin + ((c/fs)*range_corrected_rd.shape[1])/2



# az_matched_filter = azimuth_compress(az_fft_data, lambda_, prf, prf/2, doppler_fit, Rmin, Rmax, vp)
# doppler_zero = np.zeros_like(doppler_fit)
az_BW = 200




az_matched_filter = fun.azimuth_compress(range_corrected_rd, lambda_, prf, az_BW, doppler_fit, Rmin, Rmax, vp).astype(np.complex64)




In [ ]:

print(Rmax)
print(azR)
print(vp)

In [ ]:
plt.figure()

plt.imshow(np.angle(az_matched_filter), interpolation='none', vmin=-np.pi, vmax=np.pi, cmap='hsv')

plt.show()

In [ ]:
# az_matched_filter = azimuth_compress(range_corrected_rd, lambda_, prf, doppler_centroids, Rmin, Rmax, vp, 5)

# focused_rd = np.fft.fftshift(np.fft.fft(zero_doppler, axis=0), axes=0)*az_matched_filter
focused_rd = (range_corrected_rd*az_matched_filter).astype(np.complex64)
# down = fun.frequency_convert(focused_rd, doppler_fit, prf, 0)


# del(az_matched_filter)
# del(range_corrected_rd)
# focused_rd = az_fft_data*az_matched_filter
print(focused_rd.dtype)

In [ ]:
# def frequency_convert_az(data, fLO, fs):
#     #####
#     # Requires complex data
#     # fLO is freq of LO to mix with
#     # fs is sample frequency of data
#     # direction is direction of conversion - 'up' or 'down'
#     #####

#     samps = data.shape[0]
#     tmin = 0
#     tmax = samps/fs
#     n = len(fLO)

#     # fLO = fLO.reshape((n, -1))
#     fLO = fLO.reshape((-1, n))

#     t = np.linspace(tmin, tmax, samps).reshape((samps, -1))  ## Make time array
#     ft = t.dot(fLO)

#     lo = np.exp(-1j*2*np.pi*ft)
#     data = data*lo

#     return data

In [ ]:
# down = frequency_convert_az(focused_rd, doppler_fit, prf)


In [ ]:
# plt.figure()

# plt.imshow(np.angle(az_matched_filter), cmap='hsv', interpolation='None')

# plt.colorbar()
# plt.show()

In [ ]:
# del(range_corrected_rd)
# del(az_matched_filter)
focused = np.fft.ifft(focused_rd, axis=0).astype(np.complex64)
# focused = fun.frequency_convert(focused, doppler_fit, prf, 0)
# focused = np.fft.ifft(range_corrected_rd, axis=0)

# del(focused_rd)
# focused = rolling_avg(focused, 62, 0)
print(focused.dtype)


In [ ]:
plt.figure()

plt.imshow(abs(np.fft.fft(focused, axis=0)))

plt.show()

In [ ]:
focused_multilook = fun.multilook(abs(focused), 1, 4)


In [ ]:
azR = azR/4

In [ ]:
%matplotlib widget
plt.figure()

plt.imshow(20*np.log10(abs(focused)), vmin=70, vmax=100, origin='upper', cmap='gray', interpolation='none')#, aspect='equal', extent=[Rmin/1000, Rmax/1000, azR/1000, 0])
# plt.imshow(np.angle(focused_rd[6000:7350, 7100:7500]), origin='lower', cmap='hsv', interpolation='None')
# plt.imshow(np.angle(range_corrected_rd[6000:7350, 7100:7500]), origin='lower', cmap='hsv', interpolation='None')
plt.title('C-Band Grand Mesa: March 27, 2024')
plt.xlabel('Slant Range [m]')
plt.ylabel('Along Track [m]')
plt.colorbar(label='[dB - Uncal]')

# image_file = f'CBand_NH_Test2.png'
# cwd = os.getcwd()

# plt.savefig(os.path.join(cwd, image_file), dpi=1000, bbox_inches='tight', pad_inches=0.25, transparent=False)



plt.show()

In [ ]:
# (az*over/(fs*8)) * (3e8 /2)
# (50/fs) *(3e8 /2)
0.886*2*72/0.8

In [ ]:
az = 600
r = 100

cr = focused[6400:6400+az, 9225:9225+r]
cr_fft2 = np.fft.fft2(cr)

over = 4

cr_over = np.fft.ifft2(cr_fft2, s=[over*az, over*r])



plt.figure()

plt.imshow(20*np.log10(abs(cr_over)), interpolation='none')

# plt.ylim(-10, 0)

plt.show()

In [ ]:
600 *72 * 0.001

In [ ]:
200/(fs*over) * 3e8 /2

In [ ]:
r_m = np.linspace(-15.3/2 +1.4, 15.3/2 +1.4, r*over)
az_m = np.linspace(-43.2/2, 43.2/2, az*over)

fig, ax = plt.subplots(figsize=[8, 4], nrows=1, ncols=2, sharey=True)

# ax[0].plot(20*np.log10(abs(cr_over[1630, 0:300])) - 42.5, color='k')
ax[0].plot(r_m, 20*np.log10(abs(cr_over[1000, :])) - 87.5, color='k')

ax[0].set_title('Corner Reflector Range Profile')
ax[0].set_xlabel('[m]')
ax[0].set_ylabel('Normalized Magnitude [dB]')
ax[0].set_xlim(-6, 6)
ax[0].set_ylim(-40, 0)
ax[0].grid()

ax[1].set_title('Corner Reflector Azimuth Profile')

ax[1].plot(az_m, 20*np.log10(fun.rolling_avg(abs(cr_over[:, 210]), 10)) - 87.15, color='k')
ax[1].set_xlim(-20, 20)

ax[1].set_xlabel('[m]')
ax[1].grid()







# image_file = f'CR_impulse.pdf'


# plt.savefig(os.path.join(r'C:\Users\epsutherland\Desktop\ForThesis\GM_CReflectors', image_file), dpi=1000, bbox_inches='tight', pad_inches=0.25, transparent=False)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[8, 4], nrows=1, ncols=2, sharey=True)

# ax[0].plot(20*np.log10(abs(cr_over[1630, 0:300])) - 42.5, color='k')
ax[0].plot(r_m, np.angle(cr_over[1000, :]), color='k')

ax[0].set_title('Corner Reflector Range Profile')
ax[0].set_xlabel('[m]')
ax[0].set_ylabel('Normalized Magnitude [dB]')
ax[0].set_xlim(-6, 6)
ax[0].set_ylim(-np.pi, np.pi)
ax[0].grid()

ax[1].set_title('Corner Reflector Azimuth Profile')

ax[1].plot(az_m, np.angle(cr_over[:, 210]), 10, color='k')
ax[1].set_xlim(-20, 20)

ax[1].set_xlabel('[m]')
ax[1].grid()







# image_file = f'CR_impulse.pdf'


# plt.savefig(os.path.join(r'C:\Users\epsutherland\Desktop\ForThesis\GM_CReflectors', image_file), dpi=1000, bbox_inches='tight', pad_inches=0.25, transparent=False)

plt.show()

In [ ]:
# Create first plot with y1
fig, ax1 = plt.subplots()


ax1.set_xlabel('X axis')
ax1.set_ylabel('Y1')
ax1.plot(np.angle(cr_over[1000:3000, 149]), color='k', alpha=0.1)


# Create second plot with y2
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Y2')
ax2.plot(20*np.log10(abs(cr_over[1000:3000, 149])), color='k')


# Adjust y-limits for each plot
ax1.set_ylim(-np.pi, np.pi)  # Adjust according to the range of y1
ax2.set_ylim(0, 50)  # Adjust according to the range of y2

# ax1.set_xlim(0, 600)

# plt.grid(linestyle=':', color='k')

# Show plot
plt.show()

In [ ]:
# plt.figure()

# # plt.plot(20*np.log10(corrected2[780:815, 770]))
# # plt.plot(20*np.log10(focused_multilook[4172, 9150:9350]))
# # plt.plot(np.angle(focused_multilook[4172, 9150:9350]))
# # plt.plot(20*np.log10(abs(cr_over[600:1100, 72])))
# # plt.plot(np.angle(cr_over[600:1100, 72]))
# plt.plot(20*np.log10(abs(cr_over[1630, :])))
# plt.plot(np.angle(cr_over[1630, :]))



# plt.show()

fig, ax1 = plt.subplots()


ax1.set_xlabel('X axis')
ax1.set_ylabel('Y1')
ax1.plot(np.angle(cr_over[1630, :]), color='k', alpha=0.1)


# Create second plot with y2
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Y2')
ax2.plot(20*np.log10(abs(cr_over[1630, :])), color='k')


# Adjust y-limits for each plot
ax1.set_ylim(-np.pi, np.pi)  # Adjust according to the range of y1
ax2.set_ylim(0, 50)  # Adjust according to the range of y2

ax1.set_xlim(0, 400)

plt.grid(linestyle=':', color='k')

# Show plot
plt.show()

In [ ]:
outputpath = r'C:\Users\epsutherland\Desktop\ForThesis\GM_CReflectors'
channel='xxxxxxx'
timestamp='GM_CornerReflectors'
L=1

# focused, range_res, az_res = fun.fix_slc_resolution(focused, vp, prf, fs, L, fh, fl)

# fun.write_slc(focused, outputpath, channel, timestamp, start_file, stop_file, vp, H, f0, fs, prf, az_BW, range_res, az_res)
# fun.write_doppler_fit(doppler_fit, os.path.join(outputpath, 'GM_CornerReflector_Doppler.dat'))

In [ ]:
# scale = max(abs(focused.flatten()))/2**16

In [ ]:
# fun.write_slc(focused, cwd, 'ch2_20240327T123855', start_file, stop_file, vp, H, f0, fs, prf, az_BW)

In [ ]:
# az_samples, range_samples = focused.shape

# ch0 = fun.read_SNOWWI_SLC(r'c:\Users\epsutherland\Downloads\ToLabPC\Jupyter Notebook', 'ch0_test.dat', az_samples, range_samples)
# ch2 = fun.read_SNOWWI_SLC(r'c:\Users\epsutherland\Downloads\ToLabPC\Jupyter Notebook', 'ch2_test.dat', az_samples, range_samples)


